In [1]:
import os,sys
import numpy as np
from sklearn.metrics import confusion_matrix

In [2]:
with open('/home/zhufl/Temporal-Residual-Motion-Generation/tsn-pytorch/data/ucf101_flow_val_split_1.txt', 'r') as f:
    flow_list = f.readlines()
    
with open('/home/zhufl/Temporal-Residual-Motion-Generation/tsn-pytorch/data/ucf101_rgb_val_split_1.txt', 'r') as f:
    rgb_list = f.readlines()

flow_list = [x[:-1] for x in flow_list]
rgb_list = [x[:-1] for x in rgb_list]
    
flow_dict = {k:v for v, k in enumerate(flow_list)}
print(flow_dict.get('/home/zhufl/Data/UCF101_Frame/v_TaiChi_g04_c04 173'))

convt_list = []
for k in rgb_list:
    convt_list.append(flow_dict.get(k))
    
print(len(convt_list))

None
3783


In [3]:
for idx, x in enumerate(convt_list):
    print('flow order video name {}'.format(rgb_list[idx]))
    print('rgb order video name {}'.format(flow_list[x]))
    
    if idx > 10:
        break

flow order video name /home/zhufl/Data/UCF101_Frame/v_Billiards_g06_c05 260 11
rgb order video name /home/zhufl/Data/UCF101_Frame/v_Billiards_g06_c05 260 11
flow order video name /home/zhufl/Data/UCF101_Frame/v_Nunchucks_g01_c03 234 55
rgb order video name /home/zhufl/Data/UCF101_Frame/v_Nunchucks_g01_c03 234 55
flow order video name /home/zhufl/Data/UCF101_Frame/v_ParallelBars_g02_c01 108 56
rgb order video name /home/zhufl/Data/UCF101_Frame/v_ParallelBars_g02_c01 108 56
flow order video name /home/zhufl/Data/UCF101_Frame/v_JavelinThrow_g05_c06 50 44
rgb order video name /home/zhufl/Data/UCF101_Frame/v_JavelinThrow_g05_c06 50 44
flow order video name /home/zhufl/Data/UCF101_Frame/v_BalanceBeam_g04_c02 230 4
rgb order video name /home/zhufl/Data/UCF101_Frame/v_BalanceBeam_g04_c02 230 4
flow order video name /home/zhufl/Data/UCF101_Frame/v_CricketShot_g02_c04 93 23
rgb order video name /home/zhufl/Data/UCF101_Frame/v_CricketShot_g02_c04 93 23
flow order video name /home/zhufl/Data/UCF10

In [4]:
score_file = 'flow_save_score.npz'
flow_scores = np.load(score_file)
score_file = 'rgb_save_score_2.npz'
rgb_scores = np.load(score_file)
print(flow_scores['scores1'].shape)
print(rgb_scores['scores2'].shape)
print(rgb_scores['scores3'].shape)

(3783, 10, 101)
(3783, 10, 101)
(3783, 10, 101)


In [5]:
flow_rst1 = flow_scores['scores1'] # 7x7
flow_rst2 = flow_scores['scores2'] # TSN
flow_rst3 = flow_scores['scores3'] # 14x14
print(flow_rst1.shape)
flow_label = flow_scores['label']
print(flow_label.shape)

rgb_label = rgb_scores['label']
rgb_rst1 = rgb_scores['scores1'] # 7x7
rgb_rst2 = rgb_scores['scores2'] # TSN
rgb_rst3 = rgb_scores['scores3'] # 14x14

(3783, 10, 101)
(3783,)


In [6]:
## convt rgb sequence order to flow
flow_rst1_convt = [flow_rst1[x] for x in convt_list]
flow_rst2_convt = [flow_rst2[x] for x in convt_list]
flow_rst3_convt = [flow_rst3[x] for x in convt_list]
flow_label_convt = [flow_label[x] for x in convt_list]
print(flow_label_convt[0])

11


In [7]:
rgb_rst2.shape

(3783, 10, 101)

In [8]:
## Single Mode Test
video_pred = [np.argmax(x.mean(axis=0)) for x in rgb_rst2]
print(list(set(video_pred)))

cf = confusion_matrix(rgb_label, video_pred).astype(float)
cls_cnt = cf.sum(axis=1)
cls_hit = np.diag(cf)
print 'cls_hit:'
print cls_hit
print 'cls_cnt:'
print cls_cnt
cls_acc = cls_hit / cls_cnt
print(cls_acc)
print('Accuracy {:.02f}%'.format(np.mean(cls_acc) * 100))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
cls_hit:
[40. 31. 34. 35. 30. 41. 36. 25. 37. 48. 37. 40. 34. 33. 15. 42. 44. 37.
 25. 29. 30. 38. 23. 23. 33. 45. 39. 33. 38. 32. 32. 26. 26. 28. 17. 33.
 24. 11. 39. 24. 35. 49. 34. 45. 21. 36. 26. 16. 33. 34. 31. 22. 32. 35.
 31. 13. 36. 20. 37. 41. 49. 47. 41. 28. 40. 31. 28. 40. 28. 19. 34. 19.
 25. 41. 31. 36. 33. 33. 26. 28. 32. 28. 31. 31. 39. 32. 34. 32. 38. 39.
 26. 48. 33. 29. 43. 27. 34. 32. 20. 45. 25.]
cls_cnt:
[44. 32. 41. 35. 31. 43. 43. 35. 37. 48. 38. 40. 38. 33. 30. 43. 49. 37.
 28. 36. 33. 39. 36. 49. 33. 45. 45. 34. 40. 36. 37. 37. 39. 33. 33. 45.
 28. 34. 41. 37. 35. 49. 34. 

In [9]:
## Fusion Test
## OFF paper achieves 95.50% with RGB + RGB_OFF_7x7 + FLOW + FLOW_OFF_7x7
## self implementation with 95.24%; There is 0.26% less then 95.5%;

video_pred = [
    1. *x.mean(axis=0) + 1.5 * y.mean(axis=0) + 1.6 *z.mean(axis=0) +
    1.2*m.mean(axis=0) + .8 * n.mean(axis=0) + 1.7*q.mean(axis=0) for x, y, z, m, n, q in zip(rgb_rst1, rgb_rst2, rgb_rst3, flow_rst1_convt, flow_rst2_convt, flow_rst3_convt)
]

video_pred = [np.argmax(x) for x in video_pred]

cf = confusion_matrix(rgb_label, video_pred).astype(float)
cls_cnt = cf.sum(axis=1)
cls_hit = np.diag(cf)
print 'cls_hit:'
print cls_hit
print 'cls_cnt:'
print cls_cnt
cls_acc = cls_hit / cls_cnt
print(cls_acc)
print('Accuracy {:.02f}%'.format(np.mean(cls_acc) * 100))

cls_hit:
[42. 32. 40. 35. 31. 43. 37. 35. 37. 48. 38. 40. 36. 33. 28. 43. 42. 37.
 28. 22. 33. 39. 25. 29. 33. 45. 45. 34. 34. 31. 37. 27. 39. 31. 16. 44.
 27. 27. 41. 37. 35. 49. 34. 46. 29. 40. 37. 38. 36. 34. 35. 35. 33. 44.
 33. 31. 36. 31. 43. 41. 49. 48. 42. 28. 44. 31. 28. 40. 35. 26. 36. 26.
 28. 41. 34. 36. 42. 40. 41. 30. 32. 26. 31. 37. 41. 32. 34. 33. 41. 39.
 28. 49. 38. 32. 42. 27. 35. 31. 30. 45. 33.]
cls_cnt:
[44. 32. 41. 35. 31. 43. 43. 35. 37. 48. 38. 40. 38. 33. 30. 43. 49. 37.
 28. 36. 33. 39. 36. 49. 33. 45. 45. 34. 40. 36. 37. 37. 39. 33. 33. 45.
 28. 34. 41. 37. 35. 49. 34. 46. 31. 40. 37. 38. 36. 34. 39. 37. 33. 45.
 34. 35. 37. 33. 44. 41. 49. 48. 43. 28. 44. 31. 28. 40. 35. 28. 39. 30.
 28. 41. 34. 36. 43. 43. 46. 32. 40. 28. 31. 39. 41. 32. 34. 33. 42. 39.
 28. 49. 38. 32. 43. 28. 35. 36. 35. 45. 36.]
[0.95454545 1.         0.97560976 1.         1.         1.
 0.86046512 1.         1.         1.         1.         1.
 0.94736842 1.         0.93333333 1.      

In [142]:
## Fusion Test
# Single RGB input achieves 90.11% accuracy with below ratio

video_pred = [
    1.6 *x.mean(axis=0) + 1.5 * y.mean(axis=0) + 2. *z.mean(axis=0) +
    0*m.mean(axis=0) + 0 * n.mean(axis=0) + 0 * q.mean(axis=0) for x, y, z, m, n, q in zip(rgb_rst1, rgb_rst2, rgb_rst3, flow_rst1_convt, flow_rst2_convt, flow_rst3_convt)
]

video_pred = [np.argmax(x) for x in video_pred]

cf = confusion_matrix(rgb_label, video_pred).astype(float)
cls_cnt = cf.sum(axis=1)
cls_hit = np.diag(cf)
print 'cls_hit:'
print cls_hit
print 'cls_cnt:'
print cls_cnt
cls_acc = cls_hit / cls_cnt
print(cls_acc)
print('Accuracy {:.02f}%'.format(np.mean(cls_acc) * 100))

cls_hit:
[42. 32. 39. 35. 27. 41. 37. 29. 37. 48. 37. 40. 35. 33. 25. 43. 43. 37.
 25. 26. 32. 38. 23. 22. 33. 45. 45. 34. 35. 34. 36. 19. 37. 28. 17. 34.
 23. 19. 39. 30. 35. 49. 34. 45. 29. 35. 36. 34. 36. 34. 31. 24. 33. 38.
 32. 13. 36. 26. 38. 41. 49. 48. 43. 28. 42. 31. 27. 39. 24. 23. 37. 26.
 27. 39. 29. 36. 33. 39. 38. 30. 32. 28. 30. 28. 41. 31. 34. 32. 36. 39.
 27. 48. 34. 32. 43. 27. 35. 31. 26. 45. 30.]
cls_cnt:
[44. 32. 41. 35. 31. 43. 43. 35. 37. 48. 38. 40. 38. 33. 30. 43. 49. 37.
 28. 36. 33. 39. 36. 49. 33. 45. 45. 34. 40. 36. 37. 37. 39. 33. 33. 45.
 28. 34. 41. 37. 35. 49. 34. 46. 31. 40. 37. 38. 36. 34. 39. 37. 33. 45.
 34. 35. 37. 33. 44. 41. 49. 48. 43. 28. 44. 31. 28. 40. 35. 28. 39. 30.
 28. 41. 34. 36. 43. 43. 46. 32. 40. 28. 31. 39. 41. 32. 34. 33. 42. 39.
 28. 49. 38. 32. 43. 28. 35. 36. 35. 45. 36.]
[0.95454545 1.         0.95121951 1.         0.87096774 0.95348837
 0.86046512 0.82857143 1.         1.         0.97368421 1.
 0.92105263 1.         0.83333333 

In [215]:
## Fusion Test
# Single Optical Flow input achieves 89.88% accuracy at most with below ratio

video_pred = [
    0 *x.mean(axis=0) + 0 * y.mean(axis=0) + 0 *z.mean(axis=0) +
    3.1 * m.mean(axis=0) + 1.0 * n.mean(axis=0) + 1.2 * q.mean(axis=0) for x, y, z, m, n, q in zip(rgb_rst1, rgb_rst2, rgb_rst3, flow_rst1_convt, flow_rst2_convt, flow_rst3_convt)
]

video_pred = [np.argmax(x) for x in video_pred]

cf = confusion_matrix(rgb_label, video_pred).astype(float)
cls_cnt = cf.sum(axis=1)
cls_hit = np.diag(cf)
print 'cls_hit:'
print cls_hit
print 'cls_cnt:'
print cls_cnt
cls_acc = cls_hit / cls_cnt
print(cls_acc)
print('Accuracy {:.02f}%'.format(np.mean(cls_acc) * 100))

cls_hit:
[39. 32. 39. 29. 31. 43. 32. 33. 37. 48. 38. 40. 27. 33. 27. 39. 39. 37.
 25. 14. 33. 39. 25. 30. 24. 45. 45. 34. 33. 31. 35. 27. 39. 25. 13. 45.
 25. 28. 40. 36. 35. 48. 32. 46. 28. 40. 37. 38. 36. 31. 35. 35. 32. 31.
 30. 31. 37. 26. 43. 40. 45. 46. 37. 26. 42. 31. 23. 40. 35. 27. 36. 28.
 27. 38. 34. 36. 42. 33. 26. 26. 23. 23. 29. 39. 34. 30. 34. 33. 38. 32.
 24. 44. 38. 30. 33. 26. 34. 31. 27. 44. 29.]
cls_cnt:
[44. 32. 41. 35. 31. 43. 43. 35. 37. 48. 38. 40. 38. 33. 30. 43. 49. 37.
 28. 36. 33. 39. 36. 49. 33. 45. 45. 34. 40. 36. 37. 37. 39. 33. 33. 45.
 28. 34. 41. 37. 35. 49. 34. 46. 31. 40. 37. 38. 36. 34. 39. 37. 33. 45.
 34. 35. 37. 33. 44. 41. 49. 48. 43. 28. 44. 31. 28. 40. 35. 28. 39. 30.
 28. 41. 34. 36. 43. 43. 46. 32. 40. 28. 31. 39. 41. 32. 34. 33. 42. 39.
 28. 49. 38. 32. 43. 28. 35. 36. 35. 45. 36.]
[0.88636364 1.         0.95121951 0.82857143 1.         1.
 0.74418605 0.94285714 1.         1.         1.         1.
 0.71052632 1.         0.9        0.906976

In [20]:
## Fusion of origin RGB and Flow
video_pred = [
    0. *x.mean(axis=0) + 1.1 * y.mean(axis=0) + 0 *z.mean(axis=0) +
    0 * m.mean(axis=0) + 1. * n.mean(axis=0) + 0 *q.mean(axis=0) for x, y, z, m, n, q in zip(rgb_rst1, rgb_rst2, rgb_rst3, flow_rst1_convt, flow_rst2_convt, flow_rst3_convt)
]

video_pred = [np.argmax(x) for x in video_pred]

cf = confusion_matrix(rgb_label, video_pred).astype(float)
cls_cnt = cf.sum(axis=1)
cls_hit = np.diag(cf)
print 'cls_hit:'
print cls_hit
print 'cls_cnt:'
print cls_cnt
cls_acc = cls_hit / cls_cnt
print(cls_acc)
print('Accuracy {:.02f}%'.format(np.mean(cls_acc) * 100))

cls_hit:
[43. 32. 41. 35. 31. 43. 36. 35. 37. 48. 38. 40. 35. 33. 25. 41. 43. 37.
 27. 21. 33. 39. 24. 29. 33. 45. 45. 34. 39. 35. 37. 27. 39. 31. 15. 39.
 27. 26. 41. 34. 35. 49. 34. 46. 29. 40. 37. 38. 36. 34. 36. 32. 31. 45.
 33. 27. 37. 26. 42. 41. 49. 47. 43. 28. 44. 31. 28. 40. 35. 23. 35. 26.
 25. 41. 34. 36. 43. 37. 37. 28. 34. 25. 31. 35. 37. 32. 34. 33. 41. 39.
 25. 49. 38. 32. 41. 27. 35. 34. 30. 45. 32.]
cls_cnt:
[44. 32. 41. 35. 31. 43. 43. 35. 37. 48. 38. 40. 38. 33. 30. 43. 49. 37.
 28. 36. 33. 39. 36. 49. 33. 45. 45. 34. 40. 36. 37. 37. 39. 33. 33. 45.
 28. 34. 41. 37. 35. 49. 34. 46. 31. 40. 37. 38. 36. 34. 39. 37. 33. 45.
 34. 35. 37. 33. 44. 41. 49. 48. 43. 28. 44. 31. 28. 40. 35. 28. 39. 30.
 28. 41. 34. 36. 43. 43. 46. 32. 40. 28. 31. 39. 41. 32. 34. 33. 42. 39.
 28. 49. 38. 32. 43. 28. 35. 36. 35. 45. 36.]
[0.97727273 1.         1.         1.         1.         1.
 0.8372093  1.         1.         1.         1.         1.
 0.92105263 1.         0.83333333 0.953488